In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# 데이터 준비

In [2]:
from sklearn.datasets import fetch_20newsgroups
new = fetch_20newsgroups(subset= 'all')

In [3]:
print(new.data[0])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [4]:
new.target

array([10,  3, 17, ...,  3,  1,  7])

In [5]:
new.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# 데이터 전처리

In [13]:
new_df = pd.DataFrame({'News': new.data, 'Target': new.target})
new_df

,News,Target
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4
...,...,...
18841,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13
18842,From: rdell@cbnewsf.cb.att.com (richard.b.dell...,12
18843,From: westes@netcom.com (Will Estes)\nSubject:...,3
18844,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1


In [9]:
t_dict = {idx: name for idx, name in enumerate(new.target_names)}
t_dict

{0: 'alt.atheism',
 1: 'comp.graphics',
 2: 'comp.os.ms-windows.misc',
 3: 'comp.sys.ibm.pc.hardware',
 4: 'comp.sys.mac.hardware',
 5: 'comp.windows.x',
 6: 'misc.forsale',
 7: 'rec.autos',
 8: 'rec.motorcycles',
 9: 'rec.sport.baseball',
 10: 'rec.sport.hockey',
 11: 'sci.crypt',
 12: 'sci.electronics',
 13: 'sci.med',
 14: 'sci.space',
 15: 'soc.religion.christian',
 16: 'talk.politics.guns',
 17: 'talk.politics.mideast',
 18: 'talk.politics.misc',
 19: 'talk.religion.misc'}

In [10]:
new_df['Target'] = new_df['Target'].replace(t_dict)
new_df['Target']

0                rec.sport.hockey
1        comp.sys.ibm.pc.hardware
2           talk.politics.mideast
3        comp.sys.ibm.pc.hardware
4           comp.sys.mac.hardware
                   ...           
18841                     sci.med
18842             sci.electronics
18843    comp.sys.ibm.pc.hardware
18844               comp.graphics
18845                   rec.autos
Name: Target, Length: 18846, dtype: object

In [12]:
def data_cleansing(df):
    delete_email = re.sub(r'\b[\w\+]+@[\w]+.[\w]+.[\w]+.[\w]+\b', ' ', df)
    delete_number = re.sub(r'\b|\d+|\b', ' ',delete_email)
    delete_non_word = re.sub(r'\b[\W]+\b', ' ', delete_number)
    cleaning_result = ' '.join(delete_non_word.split())
    return cleaning_result

new_df.loc[:, 'News'] = new_df['News'].apply(data_cleansing)
new_df

,News,Target
0,From Mamatha Devineni Ratnam Subject Pens fans...,rec.sport.hockey
1,From Matthew B Lawson Subject Which high perfo...,comp.sys.ibm.pc.hardware
2,From hilmi Hilmi Eren Subject Re ARMENIA SAYS ...,talk.politics.mideast
3,From Guy Dawson Subject Re IDE vs SCSI DMA and...,comp.sys.ibm.pc.hardware
4,From Alexander Samuel McDiarmid Subject driver...,comp.sys.mac.hardware
...,...,...
18841,From jim Jim Zisfein Subject Re Migraines and ...,sci.med
18842,From richard b dell Subject Re A question abou...,sci.electronics
18843,From Will Estes Subject Mounting CPU Cooler in...,comp.sys.ibm.pc.hardware
18844,From steve hcrlgw Steven Collins Subject Re Sp...,comp.graphics


# 벡터화

In [15]:
from nltk import stem
stmmer = stem.SnowballStemmer('english')
ex_t = 'looking looks looked'
[stmmer.stem(i) for i in ex_t.split()]

['look', 'look', 'look']

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [17]:
stmmer = nltk.stem.SnowballStemmer('english')

In [18]:
class S_C_V(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(S_C_V, self).build_analyzer()
        return lambda doc:(stmmer.stem(w) for i in analyzer(doc))

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
stmmer = nltk.stem.SnowballStemmer('english')

class S_T_V(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(S_T_V, self).build_analyzer()
        return lambda doc:(stmmer.stem(w) for i in analyzer(doc))

# 모델링 및 학습

In [21]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline

vectorizer = [CountVectorizer(), TfidfVectorizer(), S_C_V(), S_T_V]
algth = [MultinomialNB(), LogisticRegression()]
pp_l = []

In [22]:
import itertools
for i in list(itertools.product(vectorizer, algth)):
    pp_l.append(make_pipeline(*i))

In [23]:
n_par = [(1, 1), (1, 3)]
stopw_par = ['english']
l_case_par = [True, False]
max_par = np.linspace(0.4, 0.6, num= 6)
min_par = np.linspace(0.0, 0.0, num= 1)

In [ ]:
att = {'ngram_range': n_par, 
       'max_df': max_par, 
       'min_df': min_par,
       'lowercases': I_case_par,
       'stopw_par': stopw_par
}

vectorizer_names = ['countvectorize', 'tfidfvectorizer', 's_c_v', 's_t_v']
vectorizer_par_dict = {}

for i in vectorizer_names:
    vectorizer_par_dict[i] = {}
    
    for key, v in att.items():
        